## Using the leading Frontier LLMs via their APIs

### Setting up your keys

If you haven't done so already, you'll need to create API keys from OpenAI, Anthropic and Google

For OpenAI, visit https://openai.com/api/
For Anthropic, visit https://console.anthropic.com/
For Google, visit https://ai.google.dev/gemini-api

When you get your API keys, you need to set them as environment variables, such as described in the quickstart here:
https://platform.openai.com/docs/quickstart

Or create a file called .env in this project root directory, and set your keys there:
```
OPENAI_API_KEY=xxxx
ANTHROPIC_API_KEY=xxxx
GOOGLE_API_KEY=xxxx
````

In [ ]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic

In [ ]:
# Load environment variables in a file called .env

load_dotenv()

In [ ]:
# Connect to OpenAI, Anthropic and Google

gpt = OpenAI()
claude = anthropic.Anthropic()
google.generativeai.configure()
gemini = google.generativeai.GenerativeModel('gemini-pro')

In [ ]:
# GPT-3.5
# Temperature is between 0 and 2; higher for more random output; lower for more focused and deterministic.

completion = gpt.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[
    {"role": "system", "content": "You are great at telling jokes"},
    {"role": "user", "content": "Tell a light joke for a room full of data scientists"}
  ],
    temperature=1.0,
)
print(completion.choices[0].message.content)

In [ ]:
# GPT-4

completion = gpt.chat.completions.create(
    model='gpt-4-turbo',
    messages=[
    {"role": "system", "content": "You are great at telling jokes"},
    {"role": "user", "content": "Tell a light joke for a room full of data scientists"}
  ],
    temperature=1.0
)
print(completion.choices[0].message.content)

In [ ]:
# GPT-4o

completion = gpt.chat.completions.create(
    model='gpt-4o',
    messages=[
    {"role": "system", "content": "You are great at telling jokes"},
    {"role": "user", "content": "Tell a light joke for a room full of data scientists"}
  ],
    temperature=1.0
)
print(completion.choices[0].message.content)

In [ ]:
# Claude

message = claude.messages.create(
    model="claude-3-opus-20240229",
    max_tokens=200,
    temperature=1.0,
    system="You are great at telling jokes",
    messages=[
        {"role": "user", "content": "Tell a light joke for a room full of data scientists"},
    ],
)

print(message.content[0].text)

In [ ]:
# Gemini

response = gemini.generate_content("Tell a light joke for a room full of data scientists")
print(response.text)